# Kmeans over a set of GeoTiffs

This notebook loads a set of GeoTiffs into a **RDD** of Tiles, with each Tile being a band in the GeoTiff. Each GeoTiff file contains **SpringIndex-** or **LastFreeze-** value for one year over the entire USA.

Kmeans takes years as dimensions. Hence, the matrix has cells as rows and the years as columns. To cluster on all years, the matrix needs to be transposed. The notebook has two flavors of matrix transpose, locally by the Spark-driver or distributed using the Spark-workers. Once transposed the matrix is converted to a **RDD** of dense vectors to be used by **Kmeans** algorithm from **Spark-MLlib**. The end result is a grid where each cell has a cluster ID which is then saved into a SingleBand GeoTiff. By saving the result into a GeoTiff, the reader can plot it using a Python notebook as the one defined in the [python examples](../examples/python).

<span style="color:red">In this notebook the reader only needs to modify the variables in **Mode of Operation Setup**</span>.

## Dependencies

In [1]:
import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}

import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.{CellType, DoubleArrayTile, MultibandTile, Tile, UByteCellType}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.hadoop.io.SequenceFile.Writer
import org.apache.hadoop.io.{SequenceFile, _}
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import scala.sys.process._

//Spire is a numeric library for Scala which is intended to be generic, fast, and precise.
import spire.syntax.cfor._

## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store grid0, grid0_index, protected_extent and num_cols_rows (from grid0) into HDFS. Otherwise, these data structures are read from HDFS.
* **matrix_offline_mode**: If false it means the notebook will create a mtrix,  transposed it and save it to HDFS. Otherwise, these data structures are read from HDFS.
* **kmeans_offline_mode**: If false it means the notebook will train kmeans and run kemans and store kmeans model into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **BloomFinal** or **LeafFinal** which are multi-band (**4 bands**)
* **DamageIndex** and **LastFreeze** which are single-band and if set band_num higher, it will reset to 0

For kmeans the user can define the **number of iterations** and **number of clusters** as an inclusive range. Such range is defined using **minClusters**, **maxClusters**, and **stepClusters**. These variables will set a loop starting at **minClusters** and stopping at **maxClusters** (inclusive), iterating **stepClusters** at the time. <span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup
<a id='mode_of_operation_setup'></a>

In [2]:
var rdd_offline_mode = true
var matrix_offline_mode = true
var kmeans_offline_mode = true

//GeoTiffs to be read from "hdfs:///user/hadoop/spring-index/"
var dir_path = "hdfs:///user/hadoop/spring-index/"

var offline_dir_path = "hdfs:///user/emma/spring-index/"
var geoTiff_dir = "LeafFinal"
var band_num = 3

//Years between (inclusive) 1980 - 2015
val model_timeseries = (1980, 2015)
var model_first_year = 1989
var model_last_year = 2014

//Mask
val toBeMasked = true
val mask_path = "hdfs:///user/hadoop/usa_mask.tif"

//Kmeans number of iterations and clusters
var numIterations = 75
var minClusters = 70
var maxClusters = 70
var stepClusters = 10
var save_rdds = false
var save_grids = false
var save_kmeans_model = false

rdd_offline_mode = true
matrix_offline_mode = true
kmeans_offline_mode = true
dir_path = hdfs:///user/hadoop/spring-index/
offline_dir_path = hdfs:///user/emma/spring-index/
geoTiff_dir = LeafFinal
band_num = 3
model_timeseries = (1980,2015)
model_first_year = 1989
model_last_year = 2014
toBeMasked = true
mask_path = hdfs:///user/hadoop/usa_mask.tif
numIterations = 75
minClusters = 70
maxClusters = 70
stepClusters = 10
save_rdds = false
save_grids = false
save_kmeans_model = false


false


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### Mode of operation validation

In [3]:
var single_band = false
if (geoTiff_dir == "BloomFinal" || geoTiff_dir == "LeafFinal") {
  single_band = false
} else if (geoTiff_dir == "LastFreeze" || geoTiff_dir == "DamageIndex") {
  single_band = true
  if (band_num > 0) {
    println("Since LastFreezze and DamageIndex are single band, we will use band 0!!!")
    band_num  = 0
  }
} else {
  println("Directory unknown, please set either BloomFinal, LeafFinal, LastFreeze or DamageIndex!!!")
}

if (minClusters > maxClusters) {
  maxClusters = minClusters
  stepClusters = 1
}
if (stepClusters < 1) {
  stepClusters = 1
}

//Paths to store data structures for Offline runs
var mask_str = ""
if (toBeMasked)
  mask_str = "_mask"
var grid0_path = offline_dir_path + geoTiff_dir + "Centroid/grid0" + "_"+ band_num + mask_str
var grid0_index_path = offline_dir_path + geoTiff_dir + "Centroid/grid0_index" + "_"+ band_num + mask_str
var grids_noNaN_path = offline_dir_path + geoTiff_dir + "Centroid/grids_noNaN" + "_"+ band_num + mask_str
var metadata_path = offline_dir_path + geoTiff_dir + "Centroid/metadata" + "_"+ band_num + mask_str
var grids_matrix_path = offline_dir_path + geoTiff_dir + "Centroid/grids_matrix" + "_"+ band_num + mask_str

//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

val rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grid0_path))
val matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grids_matrix_path))

if (rdd_offline_mode != rdd_offline_exists) {
  println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + rdd_offline_exists.toString())
  rdd_offline_mode = rdd_offline_exists
}
if (matrix_offline_mode != matrix_offline_exists) {
  println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + matrix_offline_exists.toString())
  matrix_offline_mode = matrix_offline_exists
}

if (!fs.exists(new org.apache.hadoop.fs.Path(mask_path))) {
  println("The mask path: " + mask_path + " is invalid!!!")
}

//Years
//val model_years = 1980 to 2015
val model_years = model_timeseries._1 to model_timeseries._2

if (!model_years.contains(model_first_year) || !(model_years.contains(model_last_year))) {
  println("Invalid range of years for " + geoTiff_dir + ". I should be between " + model_first_year + " and " + model_last_year)
  System.exit(0)
}

var model_years_range = (model_years.indexOf(model_first_year), model_years.indexOf(model_last_year))

var num_kmeans :Int  = 1
if (minClusters != maxClusters) {
  num_kmeans = ((maxClusters - minClusters) / stepClusters) + 1
}

var kmeans_model_paths :Array[String] = Array.fill[String](num_kmeans)("")
var wssse_path :String = offline_dir_path + geoTiff_dir + "Centroid/" + numIterations +"_wssse"
var geotiff_hdfs_paths :Array[String] = Array.fill[String](num_kmeans)("")
var geotiff_tmp_paths :Array[String] = Array.fill[String](num_kmeans)("")
var numClusters_id = 0

if (num_kmeans > 1) {
  numClusters_id = 0
  cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    kmeans_model_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "Centroid/kmeans_model_" + band_num + "_" + numClusters + "_" + numIterations

    //Check if the file exists
    val kmeans_exist = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))
    if (kmeans_exist && !kmeans_offline_mode) {
      println("The kmeans model path " + kmeans_model_paths(numClusters_id) + " exists, please remove it.")
    } else if (!kmeans_exist && kmeans_offline_mode) {
      kmeans_offline_mode = false
    }

    geotiff_hdfs_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "Centroid/clusters_" + band_num + "_" + numClusters + "_" + numIterations + ".tif"
    geotiff_tmp_paths(numClusters_id) = "/tmp/clusters_" + band_num + "_" + geoTiff_dir + "_" + numClusters + "_" + numIterations + ".tif"
    if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(numClusters_id)))) {
      println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(numClusters_id) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
    }
    numClusters_id += 1
  }
  kmeans_offline_mode = false
} else {
  kmeans_model_paths(0) = offline_dir_path + geoTiff_dir + "Centroid/kmeans_model_" + band_num + "_" + minClusters + "_" + numIterations
  val kmeans_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(0)))
  if (kmeans_offline_mode != kmeans_offline_exists) {
    println("\"Kmeans\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + kmeans_offline_exists.toString())
    kmeans_offline_mode = kmeans_offline_exists
  }
  geotiff_hdfs_paths(0) = offline_dir_path + geoTiff_dir + "Centroid/clusters_" + band_num + "_" + minClusters + "_" + numIterations + ".tif"
  geotiff_tmp_paths(0) = "/tmp/clusters_" + band_num + "_" + geoTiff_dir + "_" + minClusters + "_" + numIterations + ".tif"
  if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(0)))) {
    println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(0) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
  }
}

Waiting for a Spark session to start...

"Load GeoTiffs" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Matrix" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
"Kmeans" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to false
There is already a GeoTiff with the path: hdfs:///user/emma/spring-index/LeafFinalCentroid/clusters_3_70_75.tif. Please make either a copy or move it to another location, otherwise, it will be over-written.


single_band = false
mask_str = _mask
grid0_path = hdfs:///user/emma/spring-index/LeafFinalCentroid/grid0_3_mask
grid0_index_path = hdfs:///user/emma/spring-index/LeafFinalCentroid/grid0_index_3_mask
grids_noNaN_path = hdfs:///user/emma/spring-index/LeafFinalCentroid/grids_noNaN_3_mask
metadata_path = hdfs:///user/emma/spring-index/LeafFinalCentroid/metadata_3_mask
grids_matrix_path = hdfs:///user/emma/spring-index/LeafFinalCentroid/grids_matrix_3_mask
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml


fs: org.apache.hadoop.fs...


Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml

## Functions to (de)serialize any structure into Array[Byte]

In [4]:
def serialize(value: Any): Array[Byte] = {
  val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
  val obj_out_stream = new ObjectOutputStream(out_stream)
  obj_out_stream.writeObject(value)
  obj_out_stream.close
  out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
  val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
  val value = obj_in_stream.readObject
  obj_in_stream.close
  value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we filter out here all NaN values.

In [5]:
def hadoopGeoTiffRDD(satellite_filepath :String, pattern :String): RDD[(Int, (ProjectedExtent, Tile))] = {
  val listFiles = sc.binaryFiles(satellite_filepath + "/" + pattern).sortBy(_._1).keys.collect()
  var prevRDD :RDD[(Int, (ProjectedExtent, Tile))] = sc.emptyRDD

  cfor(0)(_ < listFiles.length, _ + 1) { k =>
    val filePath :String = listFiles(k)
    val kB = sc.broadcast(k)
    val currRDD = sc.hadoopGeoTiffRDD(filePath).map(m => (kB.value, m))
    prevRDD = currRDD.union(prevRDD)
    //kB.destroy()
  }
  prevRDD.sortBy(_._1)
}

hadoopGeoTiffRDD: (satellite_filepath: String, pattern: String)org.apache.spark.rdd.RDD[(Int, (geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile))]


In [6]:
def hadoopMultibandGeoTiffRDD(satellite_filepath :String, pattern :String): RDD[(Int, (ProjectedExtent, MultibandTile))] = {
  val listFiles = sc.binaryFiles(satellite_filepath + "/" + pattern).sortBy(_._1).keys.collect()
  var prevRDD :RDD[(Int,(ProjectedExtent, MultibandTile))] = sc.emptyRDD

  cfor(0)(_ < listFiles.length, _ + 1) { k =>
    val filePath :String = listFiles(k)
    val kB = sc.broadcast(k)
    val currRDD = sc.hadoopMultibandGeoTiffRDD(filePath).map(m => (kB.value,m))
    prevRDD = currRDD.union(prevRDD)
    //kB.destroy()
  }
  prevRDD.sortBy(_._1)
}

hadoopMultibandGeoTiffRDD: (satellite_filepath: String, pattern: String)org.apache.spark.rdd.RDD[(Int, (geotrellis.vector.ProjectedExtent, geotrellis.raster.MultibandTile))]


In [7]:
var t0 = System.nanoTime()
//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var grid0: RDD[(Long, Double)] = sc.emptyRDD
var grid0_index: RDD[Long] = sc.emptyRDD
var grids_noNaN_RDD: RDD[(Int, Array[Double])] = sc.emptyRDD
var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var grids_RDD :RDD[(Int, Array[Double])] = sc.emptyRDD
var mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile
var grid_cells_size :Long = 0

//Load Mask
if (toBeMasked) {
  val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values
  val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
  mask_tile0 = (mask_tiles_withIndex.filter(m => m._1==0).values.collect())(0)
}

//Local variables
val pattern: String = "*.tif"
val filepath: String = dir_path + geoTiff_dir

if (rdd_offline_mode) {
  grids_noNaN_RDD = sc.objectFile(grids_noNaN_path)
  grid0 = sc.objectFile(grid0_path)
  grid0_index = sc.objectFile(grid0_index_path)

  val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
  projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
  num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
  cellT = deserialize(metadata(3)).asInstanceOf[CellType]
} else {
  if (single_band) {
    //Lets load a Singleband GeoTiffs and return RDD just with the tiles.
    var geos_RDD = hadoopGeoTiffRDD(filepath, pattern)
    geos_RDD.cache()
    var tiles_RDD :RDD[(Int, Tile)] = geos_RDD.map{ case (i,(p,t)) => (i,t)}

    //Retrive the numbre of cols and rows of the Tile's grid
    val tiles_withIndex = tiles_RDD//.zipWithIndex().map{case (e,v) => (v,e)}
    val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
    num_cols_rows = (tile0.cols,tile0.rows)
    cellT = tile0.cellType

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = geos_RDD.map{ case (i,(p,t)) => (i,p)}//.keys.zipWithIndex().map { case (e, v) => (v, e) }
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    if (toBeMasked) {
      val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
      grids_RDD = tiles_RDD.map{ case (i,m) => (i, m.localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble().filter(!_.isNaN))}
    } else {
      grids_RDD = tiles_RDD.map{ case (i,m) => (i, m.toArrayDouble().filter(!_.isNaN))}
    }
  } else {
    //Lets load Multiband GeoTiffs and return RDD just with the tiles.
    val geos_RDD = hadoopMultibandGeoTiffRDD(filepath, pattern)
    geos_RDD.cache()
    val tiles_RDD = geos_RDD.map{ case (i,(p,t)) => (i,t)}

    //Retrive the numbre of cols and rows of the Tile's grid
    val tiles_withIndex = tiles_RDD//.zipWithIndex().map{case (e,v) => (v,e)}
    val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
    num_cols_rows = (tile0.cols,tile0.rows)
    cellT = tile0.cellType

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = geos_RDD.map{ case (i,(p,t)) => (i,p)}//.keys.zipWithIndex().map { case (e, v) => (v, e) }
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect()) (0)

    //Lets read the average of the Spring-Index which is stored in the 4th band
    val band_numB :Broadcast[Int] = sc.broadcast(band_num)
    if (toBeMasked) {
      val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
      grids_RDD = tiles_RDD.map{ case (i,m) => (i, m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())}
    } else {
      grids_RDD = tiles_RDD.map{ case (i,m) => (i, m.band(band_numB.value).toArrayDouble())}
    }
  }

  //Get Index for each Cell
  val grids_withIndex = grids_RDD

  if (toBeMasked) {
    grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => m._1 != -1000.0).map { case (v, i) => (i) }
  } else {
    //Dense vector
    //.filter(m => !m._1.isNaN).map { case (v, i) => (i) }
    //Sparse Vector
    grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i) }
  }
  //Get the Tile's grid
  grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

  //Lets filter out NaN
  if (toBeMasked) {
    grids_noNaN_RDD = grids_RDD.map{ case (i,m) => (i,m.filter(m => m != -1000.0))}
  } else {
    //Dense Vector
    grids_noNaN_RDD = grids_RDD

    //Parse Vector
    //grids_noNaN_RDD = grids_RDD.map(m => m.filter(!_.isNaN))
  }

  //Store data in HDFS
  if (save_rdds) {
    grid0.saveAsObjectFile(grid0_path)
    grid0_index.saveAsObjectFile(grid0_index_path)
    grids_noNaN_RDD.saveAsObjectFile(grids_noNaN_path)
  }

  val grids_noNaN_RDD_withIndex = grids_noNaN_RDD//.zipWithIndex().map { case (e, v) => (v, e) }
  val mod_year_diff = model_first_year-model_timeseries._1
  val mod_year_diffB = sc.broadcast(mod_year_diff)
  grids_noNaN_RDD = grids_noNaN_RDD_withIndex.filterByRange(model_years_range._1, model_years_range._2).map{ case(i,a) => (i-(mod_year_diffB.value),a)}
  
  if (save_rdds) {
    val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
    Writer.file(metadata_path),
    Writer.keyClass(classOf[IntWritable]),
    Writer.valueClass(classOf[BytesWritable])
    )

    writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
    writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
    writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
    writer.append(new IntWritable(4), new BytesWritable(serialize(cellT)))
    writer.hflush()
    writer.close()
  }
}
grid_cells_size = grid0_index.count().toInt
var t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 606837161690ns                                                    


t0 = 19501150946354
projected_extent = ProjectedExtent(Extent(-126.30312894720473, 14.29219617034159, -56.162671563152486, 49.25462702827337),geotrellis.proj4.CRS$$anon$3@41d0d1b7)
grid0 = MapPartitionsRDD[185] at map at <console>:178
grid0_index = MapPartitionsRDD[180] at map at <console>:170
grids_noNaN_RDD = MapPartitionsRDD[188] at map at <console>:201
num_cols_rows = (7808,3892)
cellT = float64raw
grids_RDD = MapPartitionsRDD[174] at map at <console>:160
mask_tile0 = FloatRawArrayTile([F@725574f1,7808,3892)


grid...


FloatRawArrayTile([F@725574f1,7808,3892)

## Matrix

We need to do a Matrix transpose to have clusters per cell and not per year. With a GeoTiff representing a single year, the loaded data looks liks this:
```
bands_RDD.map(s => Vectors.dense(s)).cache()

//The vectors are rows and therefore the matrix will look like this:
[
Vectors.dense(0.0, 1.0, 2.0),
Vectors.dense(3.0, 4.0, 5.0),
Vectors.dense(6.0, 7.0, 8.0),
Vectors.dense(9.0, 0.0, 1.0)
]
```

To achieve that we convert the **RDD[Vector]** into a distributed Matrix, a [**CoordinateMatrix**](https://spark.apache.org/docs/latest/mllib-data-types.html#coordinatematrix), which as a **transpose** method.

In [8]:
t0 = System.nanoTime()
//Global variables
var grids_matrix: RDD[Vector] = sc.emptyRDD
val grid_cells_sizeB = sc.broadcast(grid_cells_size)

if (matrix_offline_mode) {
  grids_matrix = sc.objectFile(grids_matrix_path)
} else {
  //Dense Vector
  //val mat :RowMatrix = new RowMatrix(grids_noNaN_RDD.map(m => Vectors.dense(m)))
  //Sparse Vector
  val indRowMat :IndexedRowMatrix = new IndexedRowMatrix(grids_noNaN_RDD.map{ case (i, m) => (i,m.zipWithIndex)}.map{ case (i,m) => (i,m.filter(!_._1.isNaN))}.map{ case (i,m) =>  new IndexedRow(i.toLong, Vectors.sparse(grid_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1)))})
  grids_matrix = indRowMat.toCoordinateMatrix().transpose().toIndexedRowMatrix().rows.sortBy(_.index).map(_.vector)
  if (save_grids) {
    grids_matrix.saveAsObjectFile(grids_matrix_path)
  }
}
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 25:====================================================>(994 + 6) / 1000]Elapsed time: 268104028665ns


t0 = 20112995548764
grids_matrix = MapPartitionsRDD[204] at map at <console>:82
grid_cells_sizeB = Broadcast(88)
t1 = 20381099577429


20381099577429

## Kmeans

We use Kmeans from Sparl-MLlib. The user should only modify the variables on Kmeans setup.

### Kmeans Training

In [9]:
t0 = System.nanoTime()
//Global variables
var kmeans_models :Array[KMeansModel] = new Array[KMeansModel](num_kmeans)
var wssse_data :List[(Int, Int, Double)] = List.empty

if (kmeans_offline_mode) {
  numClusters_id = 0
  cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
      println("One of the files does not exist, we will abort!!!")
      System.exit(0)
    } else {
      kmeans_models(numClusters_id) = KMeansModel.load(sc, kmeans_model_paths(numClusters_id))
    }
    numClusters_id += 1
  }
  val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
  wssse_data  = wssse_data_RDD.collect().toList
} else {
  numClusters_id = 0
  if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
    val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
    wssse_data  = wssse_data_RDD.collect().toList
  }
  grids_matrix.cache()
  cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    println(numClusters)
    kmeans_models(numClusters_id) = {
      KMeans.train(grids_matrix, numClusters, numIterations)
    }

    // Evaluate clustering by computing Within Set Sum of Squared Errors
    val WSSSE = kmeans_models(numClusters_id).computeCost(grids_matrix)
    println("Within Set Sum of Squared Errors = " + WSSSE)

    wssse_data = wssse_data :+ (numClusters, numIterations, WSSSE)

    //Save kmeans model
    if (save_kmeans_model) {
      if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
        kmeans_models(numClusters_id).save(sc, kmeans_model_paths(numClusters_id))
      }
    }
    numClusters_id += 1

    if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
      println("We will delete the wssse file")
      try { fs.delete(new org.apache.hadoop.fs.Path(wssse_path), true) } catch { case _ : Throwable => { } }
    }

    println("Lets create it with the new data")
    sc.parallelize(wssse_data, 1).saveAsObjectFile(wssse_path)
  }

  //Un-persist it to save memory
  grids_matrix.unpersist()

}
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

70
Within Set Sum of Squared Errors = 4.774033917427731E9                          
We will delete the wssse file
Lets create it with the new data
Elapsed time: 619859700965ns


t0 = 20383581272250
kmeans_models = Array(org.apache.spark.mllib.clustering.KMeansModel@320b1ea1)
wssse_data = List((70,75,7.124371401253984E9), (70,75,4.733514498278134E9), (70,75,4.737721105803598E9), (70,75,4.774033917427731E9))
t1 = 21003440973215


21003440973215

### Inspect WSSSE

In [10]:
t0 = System.nanoTime()
//current
println(wssse_data)

//from disk
if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
    var wssse_data_tmp :RDD[(Int, Int, Double)] = sc.objectFile(wssse_path)//.collect()//.toList
    println(wssse_data_tmp.collect().toList)    
}
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

List((70,75,7.124371401253984E9), (70,75,4.733514498278134E9), (70,75,4.737721105803598E9), (70,75,4.774033917427731E9))
List((70,75,7.124371401253984E9), (70,75,4.733514498278134E9), (70,75,4.737721105803598E9), (70,75,4.774033917427731E9))
Elapsed time: 208855881ns


t0 = 21005569873015
t1 = 21005778728896


21005778728896

### Run Kmeans clustering

Run Kmeans and obtain the clusters per each cell.

In [11]:
t0 = System.nanoTime()
//Cache it so kmeans is more efficient
grids_matrix.cache()

var kmeans_res: Array[RDD[Int]] = Array.fill(num_kmeans)(sc.emptyRDD)
var kmeans_centroids: Array[Array[Double]] = Array.fill(num_kmeans)(Array.emptyDoubleArray)
numClusters_id = 0
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
  kmeans_res(numClusters_id) = kmeans_models(numClusters_id).predict(grids_matrix)
  kmeans_centroids(numClusters_id) = kmeans_models(numClusters_id).clusterCenters.map(m => m(0))
  numClusters_id += 1
}

//Un-persist it to save memory
grids_matrix.unpersist()
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 48229846ns


t0 = 21006739688832
kmeans_res = Array(MapPartitionsRDD[381] at map at KMeansModel.scala:69)
kmeans_centroids = Array(Array(106.36029390042667, 33.05716826012655, 110.99789751969773, 19.447235458828004, 125.40546465618651, 72.40822229108564, 28.22730398355991, 79.45351168182628, 90.98789344163555, 64.57786634085049, 66.42400418690909, 95.8789380406569, 82.42044854514631, 110.1056484667821, 102.4171556148495, 71.1186098221191, 90.86955661613788, 96.88236399511416, 24.951751359569617, 97.44776888461406, 28.962192520898412, 11.607990506329115, 113.13738915240256, 119.32364727871762, 105.68733458822197, 87.00094283842307, 114.38016092687018, 39.31449497259903, 88.16851740184087, 112.1231527591443, 59.95153839130832, 140.71947...


[[106.36029390042667, 33.05716826012655, 110.99789751969773, 19.447235458828004, 125.40546465618651, 72.40822229108564, 28.22730398355991, 79.45351168182628, 90.98789344163555, 64.57786634085049, 66.42400418690909, 95.8789380406569, 82.42044854514631, 110.1056484667821, 102.4171556148495, 71.1186098221191, 90.86955661613788, 96.88236399511416, 24.951751359569617, 97.44776888461406, 28.962192520898412, 11.607990506329115, 113.13738915240256, 119.32364727871762, 105.68733458822197, 87.00094283842307, 114.38016092687018, 39.31449497259903, 88.16851740184087, 112.1231527591443, 59.95153839130832, 140.71947539221563, 105.27767996965247, 84.44325802705032, 30.10144190853793, 154.0006105816454, 97.04787116699758, 51.42991996716602, 82.76604027421963, 74.37568311709983, 49.884330521027884, 83.72865866678055, 120.14724252003725, 59.64349412762458, 82.78959832021393, 80.64640519280488, 128.80240895764896, 105.7156005719587, 106.8629379869319, 122.73698714023269, 76.56106469906702, 101.3639867279

#### Sanity test

It can be skipped, it only shows the cluster ID for the first 50 cells

In [12]:
t0 = System.nanoTime()
val kmeans_res_out = kmeans_res(0).take(150)
kmeans_res_out.foreach(print)

println(kmeans_res_out.size)
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242424242150
Elapsed time: 416618559ns


t0 = 21007850132835
kmeans_res_out = Array(42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42)
t1 = 21008266751394


21008266751394

## Build GeoTiff with Kmeans cluster_IDs

The Grid with the cluster IDs is stored in a SingleBand GeoTiff and uploaded to HDFS.

### Assign cluster ID to each grid cell and save the grid as SingleBand GeoTiff

To assign the clusterID to each grid cell it is necessary to get the indices of gird cells they belong to. The process is not straight forward because the ArrayDouble used for the creation of each dense Vector does not contain the NaN values, therefore there is not a direct between the indices in the Tile's grid and the ones in **kmeans_res** (kmeans result).

To join the two RDDS the knowledge was obtaing from a stackoverflow post on [how to perform basic joins of two rdd tables in spark using python](https://stackoverflow.com/questions/31257077/how-do-you-perform-basic-joins-of-two-rdd-tables-in-spark-using-python).

In [13]:
t0 = System.nanoTime()
numClusters_id = 0
val grid0_index_I = grid0_index.zipWithIndex().map{ case (v,i) => (i,v)}
grid0_index_I.cache()
grid0.cache()
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
  //Merge two RDDs, one containing the clusters_ID indices and the other one the indices of a Tile's grid cells
  val cluster_cell_pos = ((kmeans_res(numClusters_id).zipWithIndex().map{ case (v,i) => (i,v)}).join(grid0_index_I)).map{ case (k,(v,i)) => (v,i)}

  //Associate a Cluster_IDs to respective Grid_cell
  val grid_clusters :RDD[ (Long, (Double, Option[Int]))] = grid0.map { case (i, v) => if (v == 0.0) (i, Double.NaN) else (i, v) }.leftOuterJoin(cluster_cell_pos.map{ case (c,i) => (i.toLong, c)})

  //Convert all None to NaN
  val grid_clusters_res = grid_clusters.sortByKey(true).map{case (k, (v, c)) => if (c == None) (k, Int.MaxValue) else (k, c.get)}
  
  //Define a Tile
  val cluster_cellsID :Array[Int] = grid_clusters_res.values.collect()
  var cluster_cells :Array[Double] = Array.fill(cluster_cellsID.length)(Double.NaN)
  cfor(0)(_ < cluster_cellsID.length, _ + 1) { cellID =>
    if (cluster_cellsID(cellID) != Int.MaxValue) {
      cluster_cells(cellID) = kmeans_centroids(numClusters_id)(cluster_cellsID(cellID))
    }
  }
  val cluster_cellsD = DoubleArrayTile(cluster_cells, num_cols_rows._1, num_cols_rows._2)
  val geoTif = new SinglebandGeoTiff(cluster_cellsD, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))

  //Save to /tmp/
  GeoTiffWriter.write(geoTif, geotiff_tmp_paths(numClusters_id))

  //Upload to HDFS
  var cmd = "hadoop dfs -copyFromLocal -f " + geotiff_tmp_paths(numClusters_id) + " " + geotiff_hdfs_paths(numClusters_id)
  Process(cmd)!

  //Remove from /tmp/
  cmd = "rm -fr " + geotiff_tmp_paths(numClusters_id)
  Process(cmd)!

  numClusters_id += 1
}
grid0_index_I.unpersist()
grid0.unpersist()
t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 463:===================================================>(996 + 4) / 1000]]DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Elapsed time: 107152762405ns


t0 = 21010076661474
numClusters_id = 1
grid0_index_I = MapPartitionsRDD[383] at map at <console>:77
t1 = 21117229423879


21117229423879

# [Visualize results](plot_kmeans_clusters.ipynb) --------------- [Plot WSSE](kmeans_wsse.ipynb)